In [1]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type


import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()
#melb_df.head()
#удаление столбцов
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
#print(melb_df['Address'].nunique()) #количество уникальных значений в столбце
street_types = melb_df['Address'].apply(get_street_type)
#display(street_types)
popular_stypes =street_types.value_counts().nlargest(10).index
#print(popular_stypes)
# Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')
#еперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне,
# и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. 
# Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])
print(melb_df['StreetType'].nunique()) #pезультирующее число уникальных подтипов
#нет потребности хранить признак Address, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, 
# то оно определяется столбцами Longitude и Lattitude. Удалим его из нашей таблицы:
melb_df = melb_df.drop('Address', axis=1)

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

11


In [3]:
#Давайте определим число уникальных категорий в каждом столбце нашей таблицы melb_df. Для этого создадим вспомогательную таблицу unique_counts:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Type,3,object
1,Method,5,object
2,Regionname,8,object
3,Bathroom,9,int64
4,Rooms,9,int64
5,Car,11,int64
6,StreetType,11,object
7,Bedroom,12,int64
8,CouncilArea,33,object
9,Date,58,object


In [4]:
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Rooms          13580 non-null  int64  
 2   Type           13580 non-null  object 
 3   Price          13580 non-null  float64
 4   Method         13580 non-null  object 
 5   SellerG        13580 non-null  object 
 6   Date           13580 non-null  object 
 7   Distance       13580 non-null  float64
 8   Postcode       13580 non-null  int64  
 9   Bedroom        13580 non-null  int64  
 10  Bathroom       13580 non-null  int64  
 11  Car            13580 non-null  int64  
 12  Landsize       13580 non-null  float64
 13  BuildingArea   13580 non-null  float64
 14  YearBuilt      13580 non-null  int64  
 15  CouncilArea    12211 non-null  object 
 16  Lattitude      13580 non-null  float64
 17  Longtitude     13580 non-null  float64
 18  Region

None

In [5]:
#Сделаем преобразование столбцов к типу данных category:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Suburb         13580 non-null  object  
 1   Rooms          13580 non-null  int64   
 2   Type           13580 non-null  category
 3   Price          13580 non-null  float64 
 4   Method         13580 non-null  category
 5   SellerG        13580 non-null  object  
 6   Date           13580 non-null  object  
 7   Distance       13580 non-null  float64 
 8   Postcode       13580 non-null  int64   
 9   Bedroom        13580 non-null  int64   
 10  Bathroom       13580 non-null  int64   
 11  Car            13580 non-null  int64   
 12  Landsize       13580 non-null  float64 
 13  BuildingArea   13580 non-null  float64 
 14  YearBuilt      13580 non-null  category
 15  CouncilArea    12211 non-null  category
 16  Lattitude      13580 non-null  float64 
 17  Longtitude     13580 non-null  

None

In [8]:
#У типа данных category есть свой специальный аксесcор cat, который позволяет получать информацию о своих значениях и преобразовывать их. 
# Например, с помощью атрибута этого аксессора categories мы можем получить список уникальных категорий в столбце Regionname:
print(melb_df['Regionname'].cat.categories)
#каким образом столбец кодируется в виде чисел в памяти компьютера. Для этого можно воспользоваться атрибутом codes:
display(melb_df['Regionname'].cat.codes)
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')


0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']